In [30]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import math
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import datasets
from random import *


# Datasets definition
# -------------------------------------------------------------------------------------------------------
digits = datasets.load_digits()
X_digits = digits.data
Y_digits = digits.target

indAll = np.arange(len(Y_digits))
np.random.shuffle(indAll)

testToTrainRatio = 0.8
trainX=X_digits[0:int(testToTrainRatio*len(indAll))]
testX=X_digits[int(testToTrainRatio*len(indAll)):]
trainY=Y_digits[0:int(testToTrainRatio*len(indAll))]
testY=Y_digits[int(testToTrainRatio*len(indAll)):]

trainingDataset=X_digits
target=Y_digits
testingDataset=X_digits
testingTarget=Y_digits

numberOfLayerNodes=[64,32,1]
learningCoeffitient=0.01

weights = np.zeros((len(numberOfLayerNodes), np.amax(numberOfLayerNodes),np.amax(numberOfLayerNodes)))
nodes = np.zeros((len(numberOfLayerNodes), np.amax(numberOfLayerNodes)))
weightedSums = np.zeros((len(numberOfLayerNodes), np.amax(numberOfLayerNodes)))
errors = np.zeros((len(numberOfLayerNodes), np.amax(numberOfLayerNodes)))
rowError = 0
epochError = 0
currentDatasetRow = 0
                 
    
def populateWeightsWithRandom():
    global weights
    for z in range(1, len(numberOfLayerNodes)):
        for y in range(0, len(weights[0])):
            for x in range(0, len(weights[0][0])):
                weights[z][y][x] = random() * 2 - 1    
    
    
def calculateHiddenLayer(layerNumber):
    ws = 0
    for x in range(0,numberOfLayerNodes[layerNumber]):
        ws = 0
        for px in range(0, numberOfLayerNodes[layerNumber - 1]):
            ws = ws + nodes[layerNumber - 1][px] * weights[layerNumber][x][px]
            weightedSums[layerNumber][x] = ws
            nodes[layerNumber][x] = activationFunction(ws)

def activationFunction(ws):
    value = 0
    value = 1 / (1 + math.exp(-ws))
    return value                
       

def derivative(value):
    value = activationFunction(value) - math.pow(activationFunction(value), 2)
    return value
    
    
def calculateOutput():
    ws = 0
    sumOutputs = 0
    layerNumber = len(numberOfLayerNodes)-1
    for x in range(0, numberOfLayerNodes[layerNumber]):
        ws = 0
        for px in range(0, numberOfLayerNodes[layerNumber - 1]):
            ws = ws + nodes[layerNumber - 1][px] * weights[layerNumber][x][px]

        nodes[layerNumber][x] = ws
        weightedSums[layerNumber][x] = ws
        sumOutputs = sumOutputs + math.exp(nodes[layerNumber][x])

    for x in range(0, numberOfLayerNodes[layerNumber]):
        nodes[layerNumber][x] = math.exp(nodes[layerNumber][x]) / sumOutputs

            
def calculateErrors():
    rowError=0
    layerNumber = len(numberOfLayerNodes)-1
    for x in range(0, numberOfLayerNodes[layerNumber]):
        errors[layerNumber][x] = derivative(weightedSums[layerNumber][x])* (target[currentDatasetRow][x] - nodes[layerNumber][x])
        rowError=rowError+math.sqrt(math.pow(target[currentDatasetRow][x] - nodes[layerNumber][x],2))

    rowError=rowError/numberOfLayerNodes[layerNumber]

    for layerNumber in range(len(numberOfLayerNodes)-2, 0, -1):
        for x in range(0, numberOfLayerNodes[layerNumber]):
            errors[layerNumber][x] = 0
            for nx in range(0, numberOfLayerNodes[layerNumber + 1]):
                errors[layerNumber][x] = errors[layerNumber + 1][nx] * weights[layerNumber + 1][nx][x]
            errors[layerNumber][x] = derivative(weightedSums[layerNumber][x]) * errors[layerNumber][x]

    
def backPropagate():
    for layerNumber in range(1, len(numberOfLayerNodes)):
        for x in range (0, numberOfLayerNodes[layerNumber]):
            for px in range(0, numberOfLayerNodes[layerNumber - 1]):
                weights[layerNumber][x][px] = weights[layerNumber][x][px] + learningCoeffitient * errors[layerNumber][x] * nodes[layerNumber - 1][px];
    

    
def training(nEpochs):
    global currentDatasetRow
    epochError=0;
    for epoch in range(0, nEpochs):
        for y in range(0, len(trainingDataset)):
            currentDatasetRow = y
            nodes[0] = trainingDataset[y]
            for layerNumber in range(1, len(numberOfLayerNodes)-1):
                calculateHiddenLayer(layerNumber)

            calculateOutput()
            calculateErrors()
            epochError=epochError+rowError
            backPropagate()

        epochError=epochError/nEpochs
        System.out.println("Error on epoch: "+epoch+" = "+epochError)
    
    
    
def classify(row):
    nodes[0] = row
    for layerNumber in range(1, len(numberOfLayerNodes)):
        calculateHiddenLayer(layerNumber)

    calculateOutput()
    
    
    
print("-----------------------")
print(weights.shape)
print(nodes.shape)
print(weightedSums.shape)
print(errors.shape)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

populateWeightsWithRandom()
print(weights)
calculateHiddenLayer(2)

training(10)
    
    


-----------------------
(3, 64, 64)
(3, 64)
(3, 64)
(3, 64)
(1437, 64)
(1437,)
(360, 64)
(360,)
[[[ 0.          0.          0.         ...,  0.          0.          0.        ]
  [ 0.          0.          0.         ...,  0.          0.          0.        ]
  [ 0.          0.          0.         ...,  0.          0.          0.        ]
  ..., 
  [ 0.          0.          0.         ...,  0.          0.          0.        ]
  [ 0.          0.          0.         ...,  0.          0.          0.        ]
  [ 0.          0.          0.         ...,  0.          0.          0.        ]]

 [[ 0.34173127 -0.04145544 -0.23402098 ...,  0.67348685 -0.12401892
   -0.7314276 ]
  [-0.29999968 -0.35485855  0.62101434 ...,  0.78282967 -0.03000102
    0.76174701]
  [ 0.92136497 -0.75556852  0.17498261 ..., -0.44571774  0.77550236
   -0.82166647]
  ..., 
  [ 0.55136175 -0.4090853   0.31382614 ..., -0.03127131  0.97098684
    0.09646351]
  [ 0.71123752  0.39748361 -0.1892202  ...,  0.26519048 -0.61307

IndexError: invalid index to scalar variable.